In [49]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
from scipy.cluster import hierarchy as sch
import pandas as pd
import numpy as np

import yfinance as yf

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from _utils.functions import *
from _utils.portfoliohandcrafiting import *


DATA_DIR = "../_databases"

DEFAULT_DATE_FORMAT = "%Y-%m-%d"


In [50]:
# --- 1. Grupos de tickers (nomes originais) ----------------------------

safe_rates = ["BUND", "US10", "EDOLLAR"]
us_german_rates = ["US10", "EDOLLAR", "BUND"]
btp_equity = ["BTP", "CAC", "DAX", "NASDAQ", "VIX"]
commodities = ["SOYBEAN", "MILKWET", "CORN", "NOK", "GOLD", "SILVER", "COPPER", "ETHANOL"]
equity = ["CAC", "DAX", "NASDAQ", "VIX"]
equity_vol = ["CAC", "NASDAQ", "VIX"]
nok_metals = ["NOK", "GOLD", "SILVER", "COPPER"]
ags_energy = ["SOYBEAN", "MILKWET", "CORN", "ETHANOL"]
precious = ["GOLD", "SILVER"]
nok_copper = ["NOK", "COPPER"]
dry_commodities = ["SOYBEAN", "CORN"]
wet_commodities = ["MILKWET", "ETHANOL"]

# --- 2. Unificar em lista única sem duplicatas -------------------------

INSTRUMENT_LIST = sorted(list(set(
    safe_rates + us_german_rates + btp_equity + commodities +
    equity + equity_vol + nok_metals + ags_energy +
    precious + nok_copper + dry_commodities + wet_commodities
)))

print("All base tickers:", INSTRUMENT_LIST)

# --- 3. Mapeamento dos ativos originais -> ETFs em USD -----------------

etf_map = {
    # --- Rates / Bonds ---
    "IEI": "BUND",         # iShares 3-7 Year Euro Govt Bond ETF (proxy for Bunds)
    "IEF": "US10",         # iShares 7-10 Year Treasury Bond ETF
    "SHY": "EDOLLAR",      # iShares 1-3 Year Treasury Bond ETF (short-term USD)
    "IITB.MI": "BTP",      # iShares MSCI Italy ETF (proxy for Italian gov debt)

    # --- Equity Indices ---
    "EWQ": "CAC",          # iShares MSCI France ETF
    "EWG": "DAX",          # iShares MSCI Germany ETF
    "QQQ": "NASDAQ",       # Invesco QQQ Trust (Nasdaq 100)
    "VIXY": "VIX",         # ProShares VIX Short-Term Futures ETF

    # --- Commodities & FX ---
    "SOYB": "SOYBEAN",     # Teucrium Soybean Fund
    "CORN": "CORN",        # Teucrium Corn Fund
    "GLD": "GOLD",         # SPDR Gold Trust
    "SLV": "SILVER",       # iShares Silver Trust
    "CPER": "COPPER",      # United States Copper Index Fund
    "FXN": "NOK",          # iShares MSCI Norway ETF (proxy for NOK exposure)
}




All base tickers: ['BTP', 'BUND', 'CAC', 'COPPER', 'CORN', 'DAX', 'EDOLLAR', 'ETHANOL', 'GOLD', 'MILKWET', 'NASDAQ', 'NOK', 'SILVER', 'SOYBEAN', 'US10', 'VIX']


In [51]:

# --- 4. Criar lista de ETFs válidos ------------------------------------

etf_tickers = sorted([v for v in etf_map.keys() if v is not None])
print("ETFs (USD) to download:", etf_tickers)

# --- 5. Download de preços ajustados -----------------------------------

data = yf.download(tickers=etf_tickers, start="2015-01-01", back_adjust=False, auto_adjust=False,multi_level_index=False,)
data = data[['Adj Close', 'Close']]
data = data.rename(columns=etf_map)

adjusted_prices = {}
current_prices = {}
for col in data['Adj Close'].columns:
    adjusted_prices[col] = data['Close'][col]
    current_prices[col] = data['Adj Close'][col]
    
df_rets = data['Adj Close'].pct_change().dropna()
df_rets

ETFs (USD) to download: ['CORN', 'CPER', 'EWG', 'EWQ', 'FXN', 'GLD', 'IEF', 'IEI', 'IITB.MI', 'QQQ', 'SHY', 'SLV', 'SOYB', 'VIXY']


[*********************100%***********************]  14 of 14 completed


Ticker,CORN,COPPER,DAX,CAC,NOK,GOLD,US10,BUND,BTP,NASDAQ,EDOLLAR,SILVER,SOYBEAN,VIX
Date,,,,,,,,,,,,,,
2015-01-05,0.024169,0.000000,-0.034824,-0.035889,-0.046578,0.015077,0.006101,0.002284,-0.004865,-0.014669,0.000000,0.025811,0.035208,0.074721
2015-01-06,-0.002581,-0.014721,-0.004178,-0.010999,-0.018445,0.011399,0.006718,0.003093,-0.000901,-0.013409,0.000473,0.021290,0.009447,0.021670
2015-01-07,-0.019593,0.000000,0.008391,0.009838,-0.011173,-0.005891,-0.000185,0.000893,-0.004378,0.012891,0.000473,0.001263,-0.003276,-0.033142
2015-01-08,-0.005279,-0.003202,0.018911,0.018213,0.026194,-0.004209,-0.004079,-0.001216,0.004074,0.019140,-0.000118,-0.013249,-0.007512,-0.063985
2015-01-09,0.010235,-0.003747,-0.007053,-0.007903,-0.004005,0.011385,0.004933,0.002923,-0.005023,-0.006583,0.000827,0.008312,0.002365,0.039062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-03,0.010118,-0.007929,0.004419,-0.004743,0.008192,0.001793,-0.000476,-0.000285,0.000000,0.004785,0.000036,-0.005681,0.014261,-0.010899
2025-11-04,-0.005565,-0.023977,-0.013933,-0.009984,-0.010625,-0.017517,0.001244,0.000922,-0.000777,-0.020298,0.000363,-0.023309,-0.012356,0.037649
2025-11-05,0.010073,0.011464,0.009668,0.005730,-0.002527,0.011564,-0.004761,-0.002762,-0.001037,0.006508,-0.000725,0.021292,0.010354,-0.038348


In [52]:
corr_matrix = correlationEstimate(df_rets.corr())

handcraft_portfolio = handcraftPortfolio(corr_matrix,False)
    
pd.DataFrame.from_dict(handcraft_portfolio.weights(),columns=['Weigths'],orient='index')

,Weigths
SILVER,0.031250
VIX,0.250000
BTP,0.062500
BUND,0.062500
COPPER,0.125000
NOK,0.062500
CAC,0.015625
NASDAQ,0.031250
EDOLLAR,0.125000
GOLD,0.031250


In [53]:
# All ETFs utilized as Instruments is in USD and ins this notebook that is the base currency
multipliers = {'BUND': 1,
               'US10': 1,
               'EDOLLAR': 1,
               'BTP': 1,
               'CAC': 1,
               'DAX': 1,
               'NASDAQ': 1,
               'VIX': 1,
               'SOYBEAN': 1,
               'CORN': 1,
               'GOLD': 1,
               'SILVER': 1,
               'COPPER': 1,
               'NOK': 1
}

risk_target_tau = 0.3
capital = 100000
idm = 2.2 # Given from table 16 in strategy four of Advanced Futures Trading Strategies - Robert Carver
instrument_weights = handcraft_portfolio.weights()

fx_series = {'BUND': 1,
               'US10': 1,
               'EDOLLAR': 1,
               'BTP': 1,
               'CAC': 1,
               'DAX': 1,
               'NASDAQ': 1,
               'VIX': 1,
               'SOYBEAN': 1,
               'CORN': 1,
               'GOLD': 1,
               'SILVER': 1,
               'COPPER': 1,
               'NOK': 1
}

fx_series = create_fx_series_given_adjusted_prices_dict(adjusted_prices_dict=adjusted_prices,fx_series=fx_series)

In [54]:
std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
        adjusted_prices=adjusted_prices,
        current_prices=current_prices,
        annualise_stdev=True,  ## can also be False if want to use daily price diff
        use_perc_returns=True,  ## can also be False if want to use daily price diff
    )

In [55]:
position_contracts_dict = calculate_position_series_given_variable_risk_for_dict(
        capital=capital,
        risk_target_tau=risk_target_tau,
        idm=idm,
        weights=instrument_weights,
        std_dev_dict=std_dev_dict,
        fx_series_dict=fx_series,
        multipliers=multipliers,
    )

In [56]:
perc_return_dict = calculate_perc_returns_for_dict(
        position_contracts_dict=position_contracts_dict,
        fx_series=fx_series,
        multipliers=multipliers,
        capital=capital,
        adjusted_prices=adjusted_prices,
    )

portfolio_return = aggregate_returns(perc_return_dict)

In [57]:
# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=portfolio_return.index,
        y=portfolio_return.cumsum()*100,
        name="Portfolio",
        line=dict(color="blue"),
    )
)

fig.update_layout(
    title="Jumbo portfolio com ETFs",
    xaxis_title="Data",
    yaxis_title="Retorno Acumulado (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas na Frequência Diária")
print(calculate_stats(portfolio_return))
print("================================")
print("Estatísticas na Frequência Anual")
print(calculate_stats(portfolio_return, freq=YEAR))
print("================================")

Estatísticas de Desempenho
Estatísticas na Frequência Diária
{'ann_mean': 0.14314541635184605, 'ann_std': 0.2681521464027102, 'sharpe': 0.5338216317570349, 'skew': 0.6444646232029412, 'avg_drawdown': 0.250688415019493, 'max_drawdown': 0.6937717822254291, 'quant_lower': 1.2171373564128365, 'quant_upper': 1.609494061984183}
Estatísticas na Frequência Anual
{'ann_mean': 36.43701507137899, 'ann_std': 5.3842044056905785, 'sharpe': 6.76739074632245, 'skew': 1.16109633287353, 'avg_drawdown': 0.07046756368144555, 'max_drawdown': 0.27339295291652965, 'quant_lower': 0.6707898403649266, 'quant_upper': 2.982135196892502}


In [58]:
def get_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )

    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )

    return adjusted_prices, current_prices

In [64]:
adjusted_prices, current_prices = get_data_dict(['sp500','us10'])
multipliers = dict(sp500=5, us10=1000)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(adjusted_prices,dict(sp500=1, us10=1))

capital = 1000000
idm = 1.5
instrument_weights = dict(sp500=0.5, us10=0.5)

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=adjusted_prices,
current_prices=current_prices,
annualise_stdev=True,  ## can also be False if want to use daily price diff
use_perc_returns=True,  ## can also be False if want to use daily price diff
)

position_contracts_dict = calculate_position_series_given_variable_risk_for_dict(
capital=capital,
risk_target_tau=risk_target_tau,
idm=idm,
weights=instrument_weights,
std_dev_dict=std_dev_dict,
fx_series_dict=fx_series_dict,
multipliers=multipliers,
)

perc_us_return_dict = calculate_perc_returns_for_dict(
position_contracts_dict=position_contracts_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=adjusted_prices,
)



port_us_return = aggregate_returns(perc_us_return_dict)
port_us_return.sort_index(inplace=True)




In [86]:
# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=port_us_return.index,
        y=port_us_return.cumsum()*100,
        name="Portfolio",
        line=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=perc_us_return_dict['sp500'].index,
        y=perc_us_return_dict['sp500'].cumsum()*100,
        name="SP500",
        line=dict(color="red"),
    )
)

fig.add_trace(
    go.Scatter(
        x=perc_us_return_dict['us10'].index,
        y=perc_us_return_dict['us10'].cumsum()*100,
        name="US10",
        line=dict(color="green"),
    )
)


fig.update_layout(
    title="50% SP500 Fut + 50% US10 Fut Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas na Frequência Diária")
print(calculate_stats(port_us_return))
print("================================")
print("Estatísticas na Frequência Anual")
print(calculate_stats(port_us_return, freq=YEAR))


Estatísticas de Desempenho
Estatísticas na Frequência Diária
{'ann_mean': 0.19695436027331834, 'ann_std': 0.21465224056953328, 'sharpe': 0.9175509174781616, 'skew': -0.928846671871127, 'avg_drawdown': 0.07652690215303384, 'max_drawdown': 0.7224035509564579, 'quant_lower': 1.51030365275067, 'quant_upper': 1.3231457803850795}
Estatísticas na Frequência Anual
{'ann_mean': 49.799611579411156, 'ann_std': 4.275256714377516, 'sharpe': 11.648332464325959, 'skew': -0.25718198652478585, 'avg_drawdown': 0.025407764464484384, 'max_drawdown': 0.541110068938341, 'quant_lower': 1.8904168934615735, 'quant_upper': 1.0406914060284809}


In [91]:
adjusted_prices, current_prices = get_data_dict(['winfut','wdofut'])
multipliers = dict(winfut=0.2, wdofut=10)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(adjusted_prices,dict(winfut=1, wdofut=1))

capital = 1000000
idm = 1.5
instrument_weights = dict(winfut=0.5, wdofut=0.5)

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=adjusted_prices,
current_prices=current_prices,
annualise_stdev=True,  ## can also be False if want to use daily price diff
use_perc_returns=True,  ## can also be False if want to use daily price diff
)

position_contracts_dict = calculate_position_series_given_variable_risk_for_dict(
capital=capital,
risk_target_tau=risk_target_tau,
idm=idm,
weights=instrument_weights,
std_dev_dict=std_dev_dict,
fx_series_dict=fx_series_dict,
multipliers=multipliers,
)

perc_return_dict = calculate_perc_returns_for_dict(
position_contracts_dict=position_contracts_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=adjusted_prices,
)



port_br_return = aggregate_returns(perc_return_dict)
port_br_return.sort_index(inplace=True)




In [92]:
# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=port_br_return.index,
        y=port_br_return.cumsum()*100,
        name="Portfolio",
        line=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=perc_return_dict['winfut'].index,
        y=perc_return_dict['winfut'].cumsum()*100,
        name="WINFUT",
        line=dict(color="red"),
    )
)

fig.add_trace(
    go.Scatter(
        x=perc_return_dict['wdofut'].index,
        y=perc_return_dict['wdofut'].cumsum()*100,
        name="WDOFUT",
        line=dict(color="green"),
    )
)


fig.update_layout(
    title="50% WINFUT Fut + 50% WDOFUT Fut Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas na Frequência Diária")
print(calculate_stats(port_br_return))
print("================================")
print("Estatísticas na Frequência Anual")
print(calculate_stats(port_br_return, freq=YEAR))


Estatísticas de Desempenho
Estatísticas na Frequência Diária
{'ann_mean': 0.058332781479505644, 'ann_std': 0.1288393344339975, 'sharpe': 0.4527559982808563, 'skew': 0.08130825271838853, 'avg_drawdown': 0.10692385559398764, 'max_drawdown': 0.4433082102768151, 'quant_lower': 1.135157749926452, 'quant_upper': 1.2258259918161132}
Estatísticas na Frequência Anual
{'ann_mean': 14.08042234855305, 'ann_std': 2.0005049381946907, 'sharpe': 7.03843418715057, 'skew': 0.9862544076184089, 'avg_drawdown': 0.0462028173916499, 'max_drawdown': 0.23618220076495372, 'quant_lower': 0.5306260805877089, 'quant_upper': 1.6664968959608055}
